Зависимости

Для БЕРТа 

In [ ]:
!pip install tensorflow-gpu==1.15.2
!pip install deeppavlov
!python -m deeppavlov install ner_ontonotes_bert_mult
!pip install pymorphy2

Всякое ~

In [ ]:
!pip install numpy
!pip install pandas
!pip install sklearn

In [ ]:
Лемматизация

In [ ]:
!pip install pymorphy2

In [30]:
import pandas as pd
import numpy as np
import pymorphy2
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

from deeppavlov import configs, build_model
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
from joblib import dump, load

Это для колаба - если файлы располагаются локально, то это не нужно выполнять

In [31]:
from google.colab import files

Примеры, в каком формате нам подавались файлы, присутствуют  в папке

In [32]:
files.upload()

Saving Переписка_По_Проблемам.xlsx to Переписка_По_Проблемам (2).xlsx
Saving Проблемы с фото в ответе.xlsx to Проблемы с фото в ответе (2).xlsx
Saving Список_Проблем.xlsx to Список_Проблем (2).xlsx


In [33]:
msg = pd.read_excel("Переписка_По_Проблемам.xlsx")
photo_in_answer = pd.read_excel("Проблемы с фото в ответе.xlsx")
troubles = pd.read_excel("Список_Проблем.xlsx")

Функция для всего препроцессинга - делается достаточно долго для больших данных, т.к лемматизация и разметка БЕРТом

> preprocess

Передаём соответственно три датафрейма: с проблемами, по поводу фото от ответчика, и с перепиской, параметр train отвечает за то, нужно ли делать колонку target со значениями предсказывемой переменной

In [34]:
def preprocess(troubles, photo_in_answer, msg, train = True):
  
  # Ставим 1 там, для каких проблем есть фото, используем id для поиска
  troubles['is_photo_answer'] = 0
  for i in range(0, len(photo_in_answer)):
    troubles['is_photo_answer'][troubles['entity_id'] ==  photo_in_answer['id'][i]] = 1

  # В данных которые нам дали, было 34 NaN по solved - это для того, чтобы не проскакивала ошибка
  troubles['solved_at'][pd.isna(troubles['solved_at'])] = troubles['created_at'][pd.isna(troubles['solved_at'])]
  # Количество дней между solved и created
  troubles['days_to_solve'] = (troubles['solved_at'] -  troubles['created_at']).dt.days

  # Заполняем колонку с сообщениями от responsibleperson
  troubles['responsibleperson_msg'] = ' '
  for i in range(0, len(msg)):

    if (msg.person__polymorphic_ctype__model[i] == 'responsibleperson') :

      troubles['responsibleperson_msg'][troubles['entity_id'] ==  msg['entity__id'][i]] += ' | ' + str(msg['description'][i])

  # Гипотезы, отобранные вручную

  troubles['HypРекомендация'] = 0
  troubles['HypЗапрос'] = 0
  troubles['HypПланировать'] = 0
  troubles['HypФото'] = 0
  troubles['HypГод'] = 0
  troubles['HypРемонт'] = 0
  troubles['HypРабота'] = 0
  troubles['HypРассмотреть'] = 0
  troubles['HypВыезд'] = 0

  # Тут делается лемматизация

  morph = pymorphy2.MorphAnalyzer()
  troubles['responsibleperson_msg'] = troubles['responsibleperson_msg'].str.lower()
  patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
  for i in range(len(troubles['responsibleperson_msg'])):

      troubles['responsibleperson_msg'][i] = re.sub(patterns,'', troubles['responsibleperson_msg'][i])
      troubles['responsibleperson_msg'][i] = troubles['responsibleperson_msg'][i].split()

      for j in range(len(troubles['responsibleperson_msg'][i])):

        troubles['responsibleperson_msg'][i][j] = troubles['responsibleperson_msg'][i][j].strip()
        # получаем соответсвующее слово
        word = morph.normal_forms(troubles['responsibleperson_msg'][i][j])

        # Заполняем гипотезы по словам для соответствующей проблемы
        troubles['HypФото'][i] = int(troubles['is_photo_answer'][i])

        if word == 'выезд':
          troubles['HypВыезд'][i] = 1

        if word == 'год':
          troubles['HypГод'][i] = 1

        if (word == 'запрос'):
          troubles['HypЗапрос'][i] = 1

        if (word == 'рассмотреть') or ( word == 'рассмотрение'):
          troubles['HypРассмотреть'][i] = 1

        if (word == 'план') or (word == 'планировать') or (word == 'запланировать') or (word == 'планироваться') or (word == 'плановый'):
          troubles['HypПланировать'][i] = 1

        if (word == 'рекомендация'):
          troubles['HypРекомендация'][i] = 1

        if (word == 'работа'):
          troubles['HypРабота'][i] = 1

        if (word == 'ремонт'):
          troubles['HypРемонт'][i] = 1

  # БЕРТ
  ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

  # Гипотезы, которые может доставать БЕРТ
  troubles['HypLocation'] = 0
  troubles['HypMoney'] = 0
  troubles['HypDate'] = 0
  troubles['HypTime'] = 0
  troubles['HypPerson'] = 0
  troubles['HypOrganization'] = 0
  troubles['HypGPE'] = 0
  troubles['HypFacility'] = 0


  for i in range(len(troubles['responsibleperson_msg'])):

      # Это делается достаточно долго, так что можно делать логи, чтобы не умереть от ожидания ///
      if (i % 100 == 0) :
        print(i)

      # Токенизируем и разбираем на составляющие
      t = sent_tokenize(str(troubles['responsibleperson_msg'][i]))
      for j in range(len(t)):
          if len(t[j]) < 512:
              NER = ner_model([t[j]])
              if 'B-LOCATION' in NER[1][0]:
                troubles['HypLocation'][i] += 1
              if 'B-MONEY' in NER[1][0]:
                troubles['HypMoney'][i] += 1
              if 'B-DATE' in NER[1][0]:
                troubles['HypDate'][i] += 1
              if 'B-TIME' in NER[1][0]:
                troubles['HypTime'][i] += 1
              if 'B-PERSON' in NER[1][0]:
                troubles['HypPerson'][i] += 1
              if 'B-ORGANIZATION' in NER[1][0]:
                troubles['HypOrganization'][i] += 1
              if 'B-GPE' in NER[1][0]:
                troubles['HypGPE'][i] += 1
              if 'B-FAC':
                troubles['HypFacility'] += 1

  # Если есть время отправки на доработку, значит - таргет 1, т.е отмазка
  # иначе 0, если NaN
  if train:
    troubles['target'] = 1
    troubles['target'][pd.isna(troubles['rework_at'])] = 0 

  return troubles

  

Пример

In [35]:
data = preprocess(troubles, photo_in_answer, msg, True)

2020-11-29 11:36:42.76 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2020-11-29 11:37:08.122 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz download because of matching hashes
2020-11-29 11:37:08.509 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]
2020-11-29 11:37:39.504 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model
0


Моделька, чтобы можно было управлять трешхолдом (начиная с какой вероятности мы можем считать, что объект относится к классу не отмазка)

In [59]:
class model:

  def __init__(self, threshold, clf):

    self.clf = clf
    self.threshold = threshold

  def predict(self, X):
    predicted = self.clf.predict_proba(X)
    y = []
    for line in predicted:
      if (line[0] > self.threshold):
        y.append(0)
      else:
        y.append(1)
    return y

  # сохранение и загрузка модельки, просто передать имя файла /// аналог pickle
  def save_model(self, filename):

    dump(self.clf, filename + '.joblib') 

  def load_model(self, filename):

     self.clf = load(filename + '.joblib') 


Это train - передаём фичи X, таргет X, базовый классификатор clf

In [51]:
def train(X, y, clf):

  # сплитим трейн/тест
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # учим базовый классифайер
  clf.fit(X_train, y_train)
  
  # будем подбирать параметры перебором
  best_param = 0.5
  best_res = 0

  # трешхолд in [0.5, 0.95)
  # так, чтобы F метрика не очень страдала, но и рекол был как можно лучше
  for treshold in [0.5 + i*0.01 for i in range(0, 45)]:

    m = model(treshold, clf)
    res = m.predict(X_test)
    fscore = f1_score(y_test, res, average='weighted')
    recall = recall_score(y_test, res)
    if (fscore > 0.88) and (recall > best_res):

      best_res = recall
      best_param = treshold


  m = model(best_param, clf)
  # параметр сохраняется в модельке, но выведем для наглядности характеристики
  print('Best param: ', best_param) # параметр, т.е на каком трешхолде достигнуто. Если вероятность для класса с не отмазками 
  # больше него, тогда относим к этому классу, иначе к другому

  # лучший рекол и все остальные метрики
  print('Best recall: ', best_res) 
  print('Accuracy %s' % accuracy_score(y_test, m.predict(X_test)))
  print('F1 score: ', f1_score(y_test, m.predict(X_test), average='weighted'))

  return m


Все фичи для модели и наш таргет

In [50]:
features = ['days_to_solve', 'HypРекомендация','HypЗапрос', 'HypПланировать', 'HypФото', 'HypГод', 'HypРемонт', 'HypРабота', 'HypРассмотреть','HypВыезд', 'HypLocation','HypMoney',	'HypDate',	'HypTime',	'HypPerson','HypOrganization',	'HypGPE',	'HypFacility']
target = 'target'

Так обучаем - получаем обратно модель

In [ ]:
m = train(data[features], data[target], GradientBoostingClassifier())

Так можем её сохранить, а потом подгрузить где нам нужно - но при этом необходимо будет объявить класс model, конечно же

In [61]:
m.save_model('Boosting')

Например загрузим вот так

In [62]:
m.load_model('Boosting')

Предскажем таргет для всех данных

In [64]:
data['Predict'] = m.predict(data[features])

In [74]:
df = pd.DataFrame({'entity_id':data['entity_id'], 'Predict':data['Predict'], 'target':data['target']})

In [77]:
df.to_excel("Predicted.xlsx")

Загрузка из коллаба, если на локальном устройство, то не нужно

In [ ]:
files.download("Predicted.xlsx")